<a href="https://colab.research.google.com/github/luciajimenezc/luciajimenezc.github.io/blob/main/tfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

programa para extraer información del boe

In [1]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import pandas as pd

# Función para generar fechas desde una fecha inicial
def generar_fechas(desde, hasta):
    fechas = []
    fecha_actual = desde
    while fecha_actual <= hasta:
        fechas.append(fecha_actual)
        fecha_actual += timedelta(days=1)
    return fechas

# Función para descargar y procesar un XML del BOE
def procesar_boe_por_fecha(fecha):
    # Construir la URL del XML
    id_boe = f"BOE-B-{fecha.year}-{fecha.timetuple().tm_yday}"
    url = f"https://www.boe.es/diario_boe/xml.php?id={id_boe}"
    print(f"Procesando: {url}")

    try:
        response = requests.get(url)
        if response.status_code == 200:
            xml_content = response.content
            # Procesar el contenido XML
            root = ET.fromstring(xml_content)

            # Extraer información deseada
            data = []
            for seccion in root.findall(".//seccion"):
                nombre_seccion = seccion.find("nombre").text if seccion.find("nombre") is not None else "Sin nombre"
                contenido = seccion.find("contenido").text if seccion.find("contenido") is not None else "Sin contenido"
                data.append({"Fecha": fecha.strftime("%Y-%m-%d"), "Sección": nombre_seccion, "Contenido": contenido})
            return data
        else:
            print(f"Error al descargar: {url} (Código {response.status_code})")
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
    return []

# Configurar rango de fechas
fecha_inicio = datetime(2024, 1, 1)
fecha_fin = datetime.now()

# Generar todas las fechas
fechas = generar_fechas(fecha_inicio, fecha_fin)

# Procesar cada fecha y guardar resultados
resultados = []
for fecha in fechas:
    resultados.extend(procesar_boe_por_fecha(fecha))

# Guardar los resultados en un archivo CSV
if resultados:
    df = pd.DataFrame(resultados)
    df.to_csv("boe_extraccion_enero_2024.csv", index=False)
    print("Datos guardados en boe_extraccion_enero_2024.csv")
else:
    print("No se encontraron datos para guardar.")


Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-1
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-2
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-3
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-4
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-5
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-6
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-7
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-8
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-9
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-10
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-11
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-12
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-13
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-14
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-15
Proc

clasificar información

In [3]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import pandas as pd

# Definir agrupaciones del BOE y palabras clave asociadas
CATEGORIAS_BOE = {
    "Disposiciones Generales": ["ley", "decreto", "reglamento", "norma", "orden ministerial"],
    "Autoridades y Personal": ["nombramiento", "cese", "personal", "funcionario", "oposición"],
    "Oposiciones y Concursos": ["oposición", "concurso", "empleo público", "convocatoria"],
    "Subvenciones, Ayudas y Becas": ["subvención", "ayuda", "beca", "financiación", "programa"],
    "Resoluciones Judiciales": ["sentencia", "resolución judicial", "tribunal", "justicia"],
    "Anuncios Oficiales": ["subasta", "licitación", "expropiación", "aviso", "anuncio"],
    "Economía y Hacienda": ["impuesto", "presupuesto", "hacienda", "fiscal", "economía"],
    "Otros": []  # Para contenido que no encaje en otras categorías
}

# Función para clasificar contenido
def clasificar_contenido(texto, categorias):
    for categoria, palabras_clave in categorias.items():
        if any(palabra.lower() in texto.lower() for palabra in palabras_clave):
            return categoria
    return "Otros"  # Categoría predeterminada para contenido no clasificado

# Función para generar fechas desde una fecha inicial
def generar_fechas(desde, hasta):
    fechas = []
    fecha_actual = desde
    while fecha_actual <= hasta:
        fechas.append(fecha_actual)
        fecha_actual += timedelta(days=1)
    return fechas

# Función para descargar y procesar un XML del BOE
def procesar_boe_por_fecha(fecha):
    # Construir la URL del XML
    id_boe = f"BOE-B-{fecha.year}-{fecha.timetuple().tm_yday}"
    url = f"https://www.boe.es/diario_boe/xml.php?id={id_boe}"
    print(f"Procesando: {url}")

    try:
        response = requests.get(url)
        if response.status_code == 200:
            xml_content = response.content
            # Procesar el contenido XML
            root = ET.fromstring(xml_content)

            # Extraer información deseada
            data = []
            for seccion in root.findall(".//seccion"):
                nombre_seccion = seccion.find("nombre").text if seccion.find("nombre") is not None else "Sin nombre"
                contenido = seccion.find("contenido").text if seccion.find("contenido") is not None else "Sin contenido"

                # Clasificar el contenido
                categoria = clasificar_contenido(contenido, CATEGORIAS_BOE)

                # Agregar al resultado
                data.append({
                    "Fecha": fecha.strftime("%Y-%m-%d"),
                    "Sección": nombre_seccion,
                    "Contenido": contenido,
                    "Categoría": categoria
                })
            return data
        else:
            print(f"Error al descargar: {url} (Código {response.status_code})")
    except Exception as e:
        print(f"Error al procesar {url}: {e}")
    return []

# Configurar rango de fechas
fecha_inicio = datetime(2024, 1, 1)
fecha_fin = datetime.now()

# Generar todas las fechas
fechas = generar_fechas(fecha_inicio, fecha_fin)

# Procesar cada fecha y guardar resultados
resultados = []
for fecha in fechas:
    resultados.extend(procesar_boe_por_fecha(fecha))

# Guardar los resultados en un archivo CSV clasificado
if resultados:
    df = pd.DataFrame(resultados)
    df = df.sort_values(by=["Categoría", "Fecha"])  # Ordenar por categoría y fecha
    df.to_csv("boe_clasificado_ordenado.csv", index=False)
    print("Datos clasificados y guardados en boe_clasificado_ordenado.csv")
else:
    print("No se encontraron datos para guardar.")


Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-1
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-2
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-3
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-4
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-5
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-6
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-7
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-8
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-9
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-10
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-11
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-12
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-13
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-14
Procesando: https://www.boe.es/diario_boe/xml.php?id=BOE-B-2024-15
Proc

visualización

In [4]:
import streamlit as st
import pandas as pd

# Cargar el archivo CSV
df = pd.read_csv("boe_clasificado_ordenado.csv")

# Crear una aplicación interactiva
st.title("Datos Clasificados del BOE")
st.dataframe(df)


ModuleNotFoundError: No module named 'streamlit'